# This notebook calculates the water well quality metric sourced from California State Water Resources Control Board:
* % of domestic wells considered “high risk” for any of the following contaminants: Nitrate, Arsenic, 1,2,3-Trichloropropane, Hexavalent Chromium, Uranium

In [1]:
import pandas as pd
import os
import sys
import boto3
import io
import geopandas as gpd
import dask_geopandas 
import pyogrio 

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_gpkg_from_directory, upload_csv_aws
from scripts.utils.write_metadata import append_metadata

In [2]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '2b_reproject/governance/natural_resource_conservation/ca_state_water_resources_board/'

pull_gpkg_from_directory(bucket_name, aws_dir)

Saved GeoPackage as 'GAMA_division_drinking_water_reproject.gpkg' locally


In [3]:
# large file, so we break it up into partitions
water_well_data = dask_geopandas.read_file('GAMA_division_drinking_water_reproject.gpkg',npartitions=5)
# let's load in one partition so we can look at the data (up to about the first 1/5 of the rows)
df = water_well_data.partitions[0].compute()
df

,gm_dataset_name,gm_well_category,gm_data_source,gm_well_id,gm_chemical_vvl,gm_chemical_name,gm_result_modifier,gm_result,gm_chemical_units,gm_reporting_limit,...,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,DDW,MUNICIPAL,DDW,CA1910039_076_076,TCLME,Chloroform (THM),<,1.000,UG/L,1.000,...,06037432401,4324.01,Census Tract 4324.01,G5020,S,1035278,26650,+34.0910085,-118.0229572,POINT (-118.02645 34.09372)
1,DDW,MUNICIPAL,DDW,CA1910039_077_077,TBME,Bromoform (THM),<,1.000,UG/L,1.000,...,06037407002,4070.02,Census Tract 4070.02,G5020,S,2938929,252981,+34.0555809,-117.9912890,POINT (-118.00289 34.05551)
2,DDW,MUNICIPAL,DDW,CA1910039_077_077,FC113,"1,1,2-Trichloro-1,2,2-Trifluoroethane (Freon 113)",<,0.005,MG/L,0.005,...,06037407002,4070.02,Census Tract 4070.02,G5020,S,2938929,252981,+34.0555809,-117.9912890,POINT (-118.00289 34.05551)
3,DDW,MUNICIPAL,DDW,CA1910039_077_077,MTBE,MTBE (Methyl-tert-butyl ether),<,3.000,UG/L,3.000,...,06037407002,4070.02,Census Tract 4070.02,G5020,S,2938929,252981,+34.0555809,-117.9912890,POINT (-118.00289 34.05551)
4,DDW,MUNICIPAL,DDW,CA1910039_076_076,BTBZN,n-Butylbenzene,<,0.500,UG/L,0.500,...,06037432401,4324.01,Census Tract 4324.01,G5020,S,1035278,26650,+34.0910085,-118.0229572,POINT (-118.02645 34.09372)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529077,DDW,MUNICIPAL,DDW,CA5403114_004_004,TCE,Trichloroethene (TCE),<,0.500,UG/L,0.500,...,06107000302,3.02,Census Tract 3.02,G5020,S,131535691,0,+36.4515644,-119.4467551,POINT (-119.47913 36.44391)
529078,DDW,MUNICIPAL,DDW,CA5410008_007_007,BTBZN,n-Butylbenzene,<,0.500,UG/L,0.500,...,06107000204,2.04,Census Tract 2.04,G5020,S,2619285,0,+36.5376452,-119.2959057,POINT (-119.29106 36.53803)
529079,DDW,MUNICIPAL,DDW,CA5410010_024_024,PCE,Tetrachloroethene (PCE),<,0.500,UG/L,0.500,...,06107003802,38.02,Census Tract 38.02,G5020,S,1901146,0,+36.0708990,-119.0166966,POINT (-119.01752 36.07246)
529080,DDW,MUNICIPAL,DDW,CA5410048_001_001,FE,Iron,=,700.000,UG/L,100.000,...,06107003401,34.01,Census Tract 34.01,G5020,S,86435522,386377,+36.0326293,-119.0935672,POINT (-119.10091 36.07135)


In [4]:
# lots of columns! Can we get rid of some to save memory?
df.columns

Index(['gm_dataset_name', 'gm_well_category', 'gm_data_source', 'gm_well_id',
       'gm_chemical_vvl', 'gm_chemical_name', 'gm_result_modifier',
       'gm_result', 'gm_chemical_units', 'gm_reporting_limit',
       'gm_samp_collection_date', 'gm_latitude', 'gm_longitude',
       'gm_well_depth_ft', 'gm_top_depth_of_screen_ft',
       'gm_bottom_depth_of_screen_ft', 'gm_cas_number', 'gm_altwell_id1',
       'gm_altwell_id2', 'gm_altwell_id3', 'src_chemical',
       'src_result_modifier', 'src_result', 'src_chemical_units',
       'src_reporting_limit', 'src_samp_collection_date',
       'src_samp_collection_time', 'src_analytical_method', 'src_lab_note',
       'src_latitude', 'src_longitude', 'src_datum', 'src_well_depth_ft',
       'src_top_depth_of_screen_ft', 'src_bottom_depth_of_screen_ft',
       'index_right', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME',
       'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT',
       'INTPTLON', 'geometry'],
      dtype='obje

In [5]:
# columns we need to calculate our metrics
subset = water_well_data[[
    'gm_well_id',
    'gm_chemical_name',
    'gm_result_modifier',
    'gm_result',
    'gm_chemical_units', 
    'gm_reporting_limit',
    'GEOID'
]]

In [6]:
# now that the dataframe is much smaller, we can load it in
subset_df = subset.compute()
subset_df

,gm_well_id,gm_chemical_name,gm_result_modifier,gm_result,gm_chemical_units,gm_reporting_limit,GEOID
0,CA1910039_076_076,Chloroform (THM),<,1.000,UG/L,1.000,06037432401
1,CA1910039_077_077,Bromoform (THM),<,1.000,UG/L,1.000,06037407002
2,CA1910039_077_077,"1,1,2-Trichloro-1,2,2-Trifluoroethane (Freon 113)",<,0.005,MG/L,0.005,06037407002
3,CA1910039_077_077,MTBE (Methyl-tert-butyl ether),<,3.000,UG/L,3.000,06037407002
4,CA1910039_076_076,n-Butylbenzene,<,0.500,UG/L,0.500,06037432401
...,...,...,...,...,...,...,...
2645404,CA3610043_017_017,Simazine,<,1.000,UG/L,1.000,06071009400
2645405,CA1910240_004_004,Perfluoro(2-ethoxyethane) sulfonic acid (PFEESA),<,2.000,NG/L,2.000,06037920115
2645406,CA5400968_001_001,"1,1 Dichloroethylene (1,1 DCE)",<,0.500,UG/L,0.500,06107000101
2645407,CA5410010_056_056,Dibromochloromethane (THM),<,1.000,UG/L,1.000,06107003602


In [36]:
unique_chemical_names = subset_df['gm_chemical_name'].unique()
unique_chemical_names

array(['Chloroform (THM)', 'Bromoform (THM)',
       '1,1,2-Trichloro-1,2,2-Trifluoroethane (Freon 113)',
       'MTBE (Methyl-tert-butyl ether)', 'n-Butylbenzene',
       'Perfluorooctanoic acid (PFOA)', 'Potassium',
       'Dichloromethane (Methylene Chloride)', '1,3,5-Trimethylbenzene',
       '4 Chlorotoluene', 'Perchlorate',
       'Trichlorofluoromethane (Freon 11)', '2 Chlorotoluene',
       '1,1-Dichloroethane (1,1 DCA)', 'Alkalinity, total',
       'Dibromochloromethane (THM)', 'Benzene', 'Vinyl Chloride',
       '1,3-Dichlorobenzene', 'Xylenes (total)',
       'Perfluorobutane sulfonic acid (PFBS)', 'Chlorobenzene',
       'Thallium', 'Perfluorodecanoic acid (PFDA)',
       '1,1,2,2 Tetrachloroethane (PCA)', 'Sulfate',
       'Bromodichloromethane (THM)', 'Perfluorohexanoic acid (PFHxA)',
       '1,2 Dichlorobenzene (1,2-DCB)', 'Tetrachloroethene (PCE)',
       'Trichloroethene (TCE)', 'Tertiary amyl methyl ether',
       '1,1,2-Trichloroethane', 'o-Xylene', 'n-Propylbenzene 

In [48]:
# looking at how many tracts are
unique_tracts = subset_df['GEOID'].unique()
print(len(unique_tracts))

2808


In [50]:
# looking at how many wells there are
unique_wells = subset_df['gm_well_id'].unique()
print(len(unique_wells))

14692


In [78]:
# Group by both GEOID and gm_well_id, then count unique combinations
count_entries_geoid_wellid = subset_df.groupby(['GEOID', 'gm_well_id']).size().reset_index(name='num_sampled_wells')
final_count_entries_geoid = count_entries_geoid_wellid.groupby('GEOID').size().reset_index(name='num_sampled_wells')

final_count_entries_geoid

,GEOID,num_sampled_wells
0,06001430102,1
1,06001433104,1
2,06001437200,1
3,06001440301,1
4,06001441100,1
...,...,...
2803,06115040902,7
2804,06115041001,12
2805,06115041002,18
2806,06115041101,17


In [85]:
# checking this specific tract in Orange county as it had 561 rows, they are all the same well
orange_county = subset_df[subset_df['GEOID'] == '06059001303']
print(orange_county.head(5))

unique_wells_orange_county = orange_county['gm_well_id'].unique()
unique_wells_orange_county

             gm_well_id             gm_chemical_name gm_result_modifier  \
5040  CA3010018_004_004                         Iron                  <   
5048  CA3010018_004_004  trans-1,2, Dichloroethylene                  <   
5049  CA3010018_004_004            Alkalinity, total                  =   
5844  CA3010018_004_004                         Iron                  <   
5885  CA3010018_004_004                Chlorobenzene                  <   

      gm_result gm_chemical_units  gm_reporting_limit        GEOID  
5040      100.0              UG/L               100.0  06059001303  
5048        0.5              UG/L                 0.5  06059001303  
5049      310.0              MG/L                 5.0  06059001303  
5844      100.0              UG/L               100.0  06059001303  
5885        0.5              UG/L                 0.5  06059001303  


array(['CA3010018_004_004'], dtype=object)

In [51]:
# summing up the count column to make sure total well count matches above
total_well_count = final_count_entries_geoid['num_sampled_wells'].sum()

print("Total count of entries:", total_well_count)

Total count of entries: 14693


In [39]:
# subsetting data to desired contaminants
list_of_contaminants = ['Nitrate as N',
                        'Arsenic',
                        '1,2,3-Trichloropropane (1,2,3 TCP)',
                        'Chromium, Hexavalent (Cr6)',
                        'Uranium']

contaminant_data = subset_df[subset_df['gm_chemical_name'].isin(list_of_contaminants)]
contaminant_data

,gm_well_id,gm_chemical_name,gm_result_modifier,gm_result,gm_chemical_units,gm_reporting_limit,GEOID
116,CA1910039_113_113,"1,2,3-Trichloropropane (1,2,3 TCP)",<,0.0000,UG/L,0.0000,06037404703
138,CA1910039_116_116,"Chromium, Hexavalent (Cr6)",=,3.7000,UG/L,0.0200,06037980035
143,CA1910039_115_115,Nitrate as N,=,4.1000,MG/L,0.1500,06037404703
164,CA1910039_114_114,Nitrate as N,=,17.0000,MG/L,1.2000,06037404703
166,CA1910039_114_114,"1,2,3-Trichloropropane (1,2,3 TCP)",<,0.0050,UG/L,0.0050,06037404703
...,...,...,...,...,...,...,...
2645353,CA1400020_001_001,"1,2,3-Trichloropropane (1,2,3 TCP)",<,0.0007,UG/L,0.0007,06027000500
2645369,CA3700909_010_010,Nitrate as N,=,1.6000,MG/L,0.4000,06073020904
2645380,CA3610039_017_017,Nitrate as N,=,5.8000,MG/L,0.2000,06071002706
2645394,CA3700923_005_005,Nitrate as N,=,5.6000,MG/L,0.4000,06073021101


In [40]:
# further filter data to entries that exceed the reporting limit
condition1 = contaminant_data['gm_result_modifier'].isin(['>'])
condition2 = contaminant_data['gm_result'] > contaminant_data['gm_reporting_limit']
# Apply filters
filtered_df = contaminant_data[condition1 | condition2]

# Display the filtered DataFrame
filtered_df

,gm_well_id,gm_chemical_name,gm_result_modifier,gm_result,gm_chemical_units,gm_reporting_limit,GEOID
138,CA1910039_116_116,"Chromium, Hexavalent (Cr6)",=,3.7,UG/L,0.02,06037980035
143,CA1910039_115_115,Nitrate as N,=,4.1,MG/L,0.15,06037404703
164,CA1910039_114_114,Nitrate as N,=,17.0,MG/L,1.20,06037404703
174,CA1910039_114_114,Nitrate as N,=,16.0,MG/L,1.60,06037404703
197,CA0800861_001_001,Nitrate as N,=,6.2,MG/L,0.10,06015000202
...,...,...,...,...,...,...,...
2645340,CA5410010_088_088,Nitrate as N,=,2.1,MG/L,0.40,06107003401
2645369,CA3700909_010_010,Nitrate as N,=,1.6,MG/L,0.40,06073020904
2645380,CA3610039_017_017,Nitrate as N,=,5.8,MG/L,0.20,06071002706
2645394,CA3700923_005_005,Nitrate as N,=,5.6,MG/L,0.40,06073021101


In [52]:
# group by both GEOID and gm_well_id, then count unique combinations
count_contaminate_geoid = filtered_df.groupby(['GEOID', 'gm_well_id']).size().reset_index(name='num_contaminated_wells')
final_count_contaminant_geoid = count_contaminate_geoid.groupby('GEOID').size().reset_index(name='num_contaminated_wells')

final_count_contaminant_geoid


,GEOID,num_contaminated_wells
0,06001430102,1
1,06001433104,1
2,06001440301,1
3,06001441200,1
4,06001441800,8
...,...,...
2664,06115040902,4
2665,06115041001,10
2666,06115041002,7
2667,06115041101,10


In [79]:
# merge total unique wells per tract and contaminated wells per tract
well_merge = pd.merge(final_count_entries_geoid, final_count_contaminant_geoid, on='GEOID', how='left')

# calculate percentage of wells contaminated per well
well_merge['percent_sampled_wells_contaminated'] = (well_merge['num_contaminated_wells'] / well_merge['num_sampled_wells']) * 100

# replace NaN values with 0, as all tracts within this df had been sampled from, so na for contaminants is truly 0
well_merge.fillna(0, inplace=True)

well_merge

,GEOID,num_sampled_wells,num_contaminated_wells,percent_sampled_wells_contaminated
0,06001430102,1,1.0,100.000000
1,06001433104,1,1.0,100.000000
2,06001437200,1,0.0,0.000000
3,06001440301,1,1.0,100.000000
4,06001441100,1,0.0,0.000000
...,...,...,...,...
2803,06115040902,7,4.0,57.142857
2804,06115041001,12,10.0,83.333333
2805,06115041002,18,7.0,38.888889
2806,06115041101,17,10.0,58.823529


In [80]:
# checking to make sure contaminated wells does not exceed total wells
check = well_merge['num_contaminated_wells'] > well_merge['num_sampled_wells']
true_rows = well_merge[check]
true_rows

,GEOID,num_sampled_wells,num_contaminated_wells,percent_sampled_wells_contaminated


In [32]:
# read in CA census tiger file
ca_tract_county = "s3://ca-climate-index/0_map_data/ca_tracts_county.csv"
ca_tract_county = gpd.read_file(ca_tract_county)
ca_tract_county = ca_tract_county.drop(columns={'field_1', 'geometry'})
ca_tract_county = ca_tract_county.rename(columns={'TRACT':'GEOID'})

In [33]:
ca_tract_county

,GEOID,COUNTYFP,County
0,06085504321,085,Santa Clara
1,06085504410,085,Santa Clara
2,06085507003,085,Santa Clara
3,06085507004,085,Santa Clara
4,06085502204,085,Santa Clara
...,...,...,...
9124,06059001303,059,Orange
9125,06059001304,059,Orange
9126,06059001401,059,Orange
9127,06013367200,013,Contra Costa


In [81]:
# merge CA census tract with final data metric df
# leave nans as there was no data for those tracts
well_merge_ca_tracts = pd.merge(ca_tract_county, well_merge, on='GEOID', how='left')
well_merge_ca_tracts

,GEOID,COUNTYFP,County,num_sampled_wells,num_contaminated_wells,percent_sampled_wells_contaminated
0,06085504321,085,Santa Clara,NaN,NaN,NaN
1,06085504410,085,Santa Clara,NaN,NaN,NaN
2,06085507003,085,Santa Clara,NaN,NaN,NaN
3,06085507004,085,Santa Clara,NaN,NaN,NaN
4,06085502204,085,Santa Clara,NaN,NaN,NaN
...,...,...,...,...,...,...
9124,06059001303,059,Orange,1.0,1.0,100.0
9125,06059001304,059,Orange,NaN,NaN,NaN
9126,06059001401,059,Orange,NaN,NaN,NaN
9127,06013367200,013,Contra Costa,NaN,NaN,NaN


In [82]:
well_merge_ca_tracts.to_csv('governance_well_quality_metric.csv', index=False)

## Function Call

In [83]:
@append_metadata
def gama_well_water_quality_upload(input_csv, export=False, varname=''):
    '''
    Uploads the water well quality metric to S3 bucket. The metric is:
    
    * Percentage of domestic wells considered “high risk” for any of the following contaminants:
      Nitrate, Arsenic, 1,2,3-Trichloropropane, Hexavalent Chromium, Uranium

    Data for this metric was sourced from the California State Water Resources Control Board: GAMA - Division of Drinking Water at
    https://data.ca.gov/dataset/ground-water-water-quality-results/resource/d2e74ace-2cf4-4baf-aadd-406280bf1c1c?inner_span=True

    Methods
    -------
    Relevant data columns were isolated, some were renamed for later merging with California tract data.
    Total number of wells sampled per census tract were retained for the percentage calculation.
    Specific contaminants (per the metric) were isolated for.
    Sample levels exceeding the 'gm_reporting_limit' column were further retained and counted per California tract.
    Total wells sampled and total entries exceeding desired contaminants were merged together per census tract.
    Number of samples exceeding the threshold were divided by number of wells sampled, then multiplied by 100 to calculate
    percentage.
    Tracts missing data were left as nan, as missing tracts were either not sampled from, or had missing data
    
    Parameters
    ----------
    input_csv: string
        csv water well quality metric data 
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI water well quality metric to AWS
        True = will upload resulting df containing CAL CRAI water well quality metric to AWS

    Script
    ------
    governance_gama_wells.ipynb

    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are stored in ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    '''
    print('Data transformation: relevant columns and contaminants were isolated and renamed.')
    print('Data transformation: data was isolated when above its predefined threshold.')
    print('Data transformation: data were then merged to California census tracts.')

    if export == True:
        bucket_name = 'ca-climate-index'
        directory = '3_fair_data/index_data'
        export_filename = [input_csv]
        upload_csv_aws(export_filename, bucket_name, directory)

    if export == False:
        print(f'{input_csv} uploaded to AWS.')
 
    '''if os.path.exists(input_csv):
        os.remove(input_csv)'''

In [84]:
input_csv = 'governance_well_quality_metric.csv'
variable = 'governance_swcrb_groundwater_quality'

gama_well_water_quality_upload(input_csv=input_csv, export=True, varname='test')